# Import <span style=color:yellow>Modules

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras.layers import (Layer,Conv2D,BatchNormalization,Activation,
                                     Add,GlobalAveragePooling2D,Dense,Input)
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

# Data <span style=color:yellow> Preprocessing

In [ ]:
train_dic = '/kaggle/input/multiclassimagedatasetairplanecar/Dataset/train'
test_dic = '/kaggle/input/multiclassimagedatasetairplanecar/Dataset/test'
batch_size = 32
image_size = 224
learing_rate = 0.001
class_name = ['airplanes', 'cars', 'ship']
class_name_1 = ['airplanes', 'cars', 'ships']
n_class = len(class_name)

## Creating <span style=color:yellow>Dataset

In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dic,
    labels='inferred',
    label_mode='categorical',
    class_names=class_name,
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(image_size,image_size),
    shuffle=True,
    seed=99,
)

In [ ]:
train_dataset

In [ ]:
test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dic,
    labels='inferred',
    label_mode='categorical',
    class_names=class_name_1,
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(image_size,image_size),
    shuffle=True,
    seed=99,
)

In [ ]:
test_dataset

## Data <span style=color:yellow>Visulization

In [ ]:
train_dataset.take(1)

In [ ]:
for image,label in train_dataset.take(1):
    for i in range(16):
        ax = plt.subplot(4,4,i+1)
        plt.imshow(image[i]/255.)
        plt.title(class_name[tf.argmax(label[i],axis = 0).numpy()])
        plt.axis("off")

# Data <span style=color:yellow>Prepration

In [ ]:
train_dataset = (train_dataset.prefetch(tf.data.AUTOTUNE))

In [ ]:
test_dataset = (test_dataset.prefetch(tf.data.AUTOTUNE))

In [ ]:
train_dataset

In [ ]:
test_dataset

# <span style=color:yellow> Models

## <span style=color:lightblue>Resnet-50

In [ ]:
class CustomConv(Layer):
    def __init__(self, kernel_size, filter_size, activation=None):
        super(CustomConv, self).__init__()
        
        self.padding = 'valid' if kernel_size > 1 else 'same'
        
        self.conv_1 = Conv2D(filters=filter_size, kernel_size=kernel_size, activation=activation, padding=self.padding)
        self.batch_1 = BatchNormalization(axis=3)
        
    def call(self, input,training=True):
        x = self.conv_1(input)
        x = self.batch_1(x,training=training)
        return x

In [ ]:
class ConvBlock(Layer):
    def __init__(self, filter_size):
        super(ConvBlock, self).__init__()

        self.conv_0 = CustomConv(kernel_size=1, filter_size=filter_size)
        self.conv_1 = CustomConv(kernel_size=1, filter_size=filter_size, activation='relu')
        self.conv_2 = CustomConv(kernel_size=3, filter_size=filter_size, activation='relu')
        self.conv_3 = CustomConv(kernel_size=1, filter_size=filter_size)

        self.activation = Activation('relu')

        self.extra_conv = Conv2D(filter_size, kernel_size=1, padding='valid')

    def call(self, input, training):
        input_x = input
        x = self.conv_1(input,training=training)
        x = self.conv_2(x,training=training)
        x = self.conv_3(x,training=training)

        input_x = self.extra_conv(input_x,training=training)

        input_x = tf.image.resize(input_x, (x.shape[1], x.shape[2]))

        x = Add()([x, input_x])

        return self.activation(x)

In [ ]:
class IdentityBlock(Layer):
    def __init__(self, filter_size):
        super(IdentityBlock, self).__init__()

        self.conv_1 = CustomConv(kernel_size=1, filter_size=filter_size, activation='relu')
        self.conv_2 = CustomConv(kernel_size=3, filter_size=filter_size, activation='relu')
        self.conv_3 = CustomConv(kernel_size=1, filter_size=filter_size)

        self.activation = Activation('relu')

        self.extra_conv = Conv2D(filter_size, kernel_size=1, padding='valid')  
    def call(self, input,training):
        input_x = input
        x = self.conv_1(input,training=training)
        x = self.conv_2(x,training=training)
        x = self.conv_3(x,training=training)

        input_x = self.extra_conv(input_x)

        input_x = tf.image.resize(input_x, (x.shape[1], x.shape[2]))

        x = Add()([x, input_x])

        return self.activation(x)


In [ ]:
class ResNet50(Model):
    def __init__(self, num_classes=n_class):
        super(ResNet50, self).__init__(name="ResNet50")
        
        self.conv_1_1 = ConvBlock(256)
        self.idnty_1_1 = IdentityBlock(256)
        self.idnty_1_2 = IdentityBlock(256)
        
        self.conv_2_1 = ConvBlock(512)
        self.idnty_2_1 = IdentityBlock(512)
        self.idnty_2_2 = IdentityBlock(512)
        self.idnty_2_3 = IdentityBlock(512)
        
        self.conv_3_1 = ConvBlock(1024)
        self.idnty_3_1 = IdentityBlock(512)
        self.idnty_3_2 = IdentityBlock(512)
        self.idnty_3_3 = IdentityBlock(512)
        self.idnty_3_4 = IdentityBlock(512)
        self.idnty_3_5 = IdentityBlock(512)
        
        self.conv_4_1 = ConvBlock(2048)
        self.idnty_4_1 = IdentityBlock(512)
        self.idnty_4_2 = IdentityBlock(512)
        
        self.pool = GlobalAveragePooling2D()
        self.dense = Dense(num_classes, activation='softmax')
        
    def call(self, input, training):
        x = self.conv_1_1(input, training=training)
        x = self.idnty_1_1(x, training=training)
        x = self.idnty_1_2(x, training=training)
        
        x = self.conv_2_1(x, training=training)
        x = self.idnty_2_1(x, training=training)
        x = self.idnty_2_2(x, training=training)
        x = self.idnty_2_3(x, training=training)
        
        x = self.conv_3_1(x, training=training)
        x = self.idnty_3_1(x, training=training)
        x = self.idnty_3_2(x, training=training)
        x = self.idnty_3_3(x, training=training)
        x = self.idnty_3_4(x, training=training)
        x = self.idnty_3_5(x, training=training)
        
        x = self.conv_4_1(x, training=training)
        x = self.idnty_4_1(x, training=training)
        x = self.idnty_4_2(x, training=training)
        
        x = self.pool(x)
        x = self.dense(x)
        
        return x


In [ ]:
resnet50 = ResNet50()
resnet50(tf.zeros([1,256,256,3]), training=False)
resnet50.summary()

# Model <span style=color:yellow>Training 

In [ ]:
loss_function = CategoricalCrossentropy()

## <span style=color:lightblue>Resnet-50

In [ ]:
resnet50.compile(
    optimizer = Adam(learning_rate = learing_rate),
    loss = loss_function,
    metrics=['accuracy']
)

In [ ]:
history = resnet50.fit(
    train_dataset,
    epochs = 10,
    verbose = 1
)

## <span style=color:lightblue>